In [ ]:
import datetime
import glob
import numpy as np
import xarray as xr
import rasterio
import rioxarray
import pandas as pd
from matplotlib import pyplot
from rasterio.warp import Resampling

In [ ]:
with rasterio.open('data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h15v05_2023.tiff') as dataset:
    lc_map = dataset.read(1)

In [ ]:
pyplot.imshow(lc_map, interpolation = 'nearest')

In [ ]:
file_list = glob.glob('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_*.tiff')
file_list.sort()

rasters_et = []
rasters_pet = []
for filename in file_list:
    date_str = filename.split('/')[-1].split('_')[-1].replace('.tiff', '')
    date = datetime.datetime.strptime(date_str, '%Y%m%d')
    et0 = rioxarray.open_rasterio(filename)\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    rasters_et.append(et0)
    # Do the same for PET, note the use of replace()
    pet0 = rioxarray.open_rasterio(filename.replace('ET_mm_8day-1', 'PET_mm_8day-1'))\
        .rename(band = 'time')\
        .assign_coords(time = [date])
    rasters_pet.append(pet0)

# Convert from [mm 8day-1] to [mm day-1]
ds_et0 = xr.concat(rasters_et, dim = 'time').to_dataset(name = 'ET') / 8.0
ds_pet0 = xr.concat(rasters_pet, dim = 'time').to_dataset(name = 'PET') / 8.0

ds_et = xr.merge([ds_et0, ds_pet0])

```
# TODO Resampling to daily time steps; we have to a

ds_et_last = ds_et.isel(time = -1).assign_coords(time = pd.Timestamp('2023-12-31'))
ds_et_daily = xr.concat([ds_et, ds_et_last], dim = 'time')\
    .resample(time = '1D', skipna = False).ffill()
ds_et_daily
```

## Computing the initial soil water content

In [ ]:
# TODO Now loading precipitation
example = rioxarray.open_rasterio('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_20230101.tiff')

# Resample the precip data to match our ET data
ds_precip = xr.open_dataset('data/processed/IMERG_precip_mm_day-1_for_study_area.nc4')\
    .rio.write_crs(example.rio.crs)
ds_precip_500m = ds_precip.rio.reproject_match(example, resampling = Resampling.bilinear)

In [ ]:
precip_total = ds_precip_500m.sum('time').precipitation

In [ ]:
# NOTE: Our best guess for the soil water content might be
#    the mean annual balance between precip and ET

swc = (precip_total - ds_et.ET.sum('time')) / 365

In [ ]:
swc.where(swc >= 0, np.nan).where(lc_map == 1, np.nan).plot()

## Field capacity, rooting depth, and crop coefficients

The critical soil water level (SWC) is given by:
$$
\text{SWC}(t) = \text{FC}\times \text{SW}_f\times \text{RD}_f(t)
$$

- Field capacity (FC) depends on soil texture; we could use a texture map to account for the heterogeneity across our study area; for simplicity, we'll assume the soils are a sandy loam, with a median field capacity (from FAO 56 Table 19) of 0.23 m$^3$ m$^{-3}$. If the maximum rooting depth of wheat is about 1.5 m (https://doi.org/10.1016/j.fcr.2016.02.013), then this corresponds to:
$$
\text{FC} = \frac{0.23\, \text{m}^3\text{m}^{-3}}{1.5\,\text{m}\times 1.0\,\text{m} \times 1.0\,\text{m}} = 0.153 \,\text{m}
$$
- According to FAO 56 (Table 22), for Spring Wheat, $\text{SW}_f = 1 - 0.55 = 0.45$.
- $\text{RD}_f$ varies over time as the crop's roots develop. For wheat in Northern Algeria, sowing generally occurs in October or November, with the crop reaching maturity between February and April. Without knowing the conditions in individual fields, we'll assume sowing occured October 15 and the crop reached full maturity on April 1. Therefore, we'll have $\text{RD}_f$ increase from 0.1 on October 15 to 1.0 on April 1.
- For the crop coefficient, we'll use values from [FAO 56's Table 12.](https://www.fao.org/4/X0490E/x0490e0b.htm#tabulated%20kc%20values) There are three coefficients, depending on crop stage (initial, middle, and end). For Spring Wheat, these values are 0.3, 1.15, and 0.25; we'll assume that these crop stages occur on October 15, April 1, and June 1, respectively.

See for a visual aid: https://www.fao.org/4/r4082e/r4082e03.htm#2.3.1%20soil%20moisture%20content

## Our first implementation

If we need to look up the DOY associated with a date, we could use:

```python
import datetime
datetime.date(2023, 6, 1).strftime('%j')
```

In [ ]:
# TODO To begin, it's helpful to design the API before we even begin to code the implementation

def water_requirement_satisfaction_index(
        sw_init, pet_t, precip_t, root_fraction_t, field_capacity = 0.153, 
        crop_coef = [0.3, 0.15, 0.25], crop_coef_doy = [288, 91, 152]):
    '''
    Parameters
    ----------
    sw_init : Number
        The initial soil water level (mm), on the first day of simulation
    pet_t : Sequence
        The potential evapotranspiration (mm day-1) for each day of simulation
    precip_t : Sequence
        The daily precipitation (mm day-1) for each day of simulation
    root_fraction_t : Sequence
        The root depth fraction, a dimensionless number between 0.0 and 1.0,
        for each day of simulation
    field_capacity : Number
        The field capacity (mm)
    crop_coef : Sequence
        The crop coefficient, a sequence of three decimal numbers
    crop_coef_doy : Sequence
        The day of year (DOY) corresponding to each crop coefficient
        in `crop_coef`, in order

    Returns
    -------
    Number
    '''
    pass

In [ ]:
def water_requirement_satisfaction_index(
        sw_init, pet_t, precip_t, root_fraction_t, field_capacity = 0.153, 
        crop_coef = [0.3, 0.15, 0.25], crop_coef_doy = [288, 91, 152]):
    '''
    Parameters
    ----------
    sw_init : Number
        The initial soil water level (mm), on the first day of simulation
    pet_t : Sequence
        The potential evapotranspiration (mm day-1) for each day of simulation
    precip_t : Sequence
        The daily precipitation (mm day-1) for each day of simulation
    root_fraction_t : Sequence
        The root depth fraction, a dimensionless number between 0.0 and 1.0,
        for each day of simulation
    field_capacity : Number
        The field capacity (mm)
    crop_coef : Sequence
        The crop coefficient, a sequence of three decimal numbers
    crop_coef_doy : Sequence
        The day of year (DOY) corresponding to each crop coefficient
        in `crop_coef`, in order
    
    Returns
    -------
    Number
    '''
    # At every time step (including t=0), compute SWC by Equation 7
    # At t=0, use sw_init in Equations 3-6 to compute AET
    # Then, for t >= 0:
    #    Compute SW by Equation 8 (i.e., use AET just calculated)
    pass